- https://lilianweng.github.io/posts/2021-05-31-contrastive/
- siamese-triplet
    - https://github.com/adambielski/siamese-triplet/tree/master
- contrastive-image-text
    - CLIP：https://github.com/huggingface/transformers/tree/main/examples/pytorch/contrastive-image-text

### constrastive learning & metric learning 

- constrastive learning
    - 一种通过对样本对进行比较来学习有用表征的方法。目标是使相似样本的表示更接近，不相似样本的表示更远离。
    - 相似不相似，更多是类别上的；
- metric learning or embedding learning
    - embedding learning：学习的是表示；
        - 传统对比学习主要关注学习数据的表示（embedding），使得相似样本的表示更接近，以便在下游任务中使用。
- learning to rank（learning from preference）
    - 从类别到偏序
- 数据集（样本对，正例、负例，或者偏序对）的构造，以及损失函数的设计；
- contrastive learning vs. reward modeling
    - contrastive learning 学到的是一个表示（representation），或者说基于表示还可进一步算距离 $\|f(x)-f(x_+)\|$；
    - Reward modelling 你最终学到的是一个 reward/value function：$r_\theta(x,y_w)$；

### CLIP (Contrastive Language–Image Pretraining)

> 相应的图像和文本在特征空间中接近，而不相关的图文对则远离

- 在训练过程中，CLIP 处理一批（batch）图文对。例如，在一个批次中有 N 个图像和 N 个对应的文本描述。模型将每个图像和文本分别编码为向量表示。然后，它计算每个图像向量和每个文本向量之间的相似度，形成一个 N x N 的相似度矩阵。
    - 对于这个矩阵中的每一行，模型希望对应的正确匹配（即第 i 个图像和第 i 个文本）的相似度最大，而其他不匹配的相似度较小。为此，CLIP 使用了一种基于交叉熵的对比损失函数，它考虑了图像到文本和文本到图像的匹配，这意味着模型同时优化图像检索文本和文本检索图像的任务。
    - 这种损失函数的设计使得模型学会在高维特征空间中，将相关的图像和文本映射到相近的位置，从而实现跨模态的检索和匹配。
- 设在一个批次中，有 N 个图像和 N 个对应的文本描述，形成 N 个匹配的图文对。
    - 令 $I_i$ 表示第 $i$ 个图像，$T_i$ 表示第 $i$ 个文本
    - 定义图像编码器 $f$ 和文本编码器 $g$，将图像和文本映射到一个共同的特征空间。（表征学习 representation learning）
    - 获取归一化的图像和文本特征向量：

        $$
        \mathbf v_i=\frac{f(I_i)}{\|f(I_t)\|}, \mathbf u_i=\frac{g(T_i)}{\|g(T_i)\|}
        $$
    - 相似度矩阵 $S$ 的元素定义为（其中，$\tau$ 是可学习的温度（temperature）参数，用于控制分布的锐化程度。）
        - 注意 $v_i$ 和 $u_i$ 维度未必一致，也许还需要一层映射，映射到一个共同的表征空间；
        $$
        s_{ij}=\frac{v_i^Tu_i}{\tau}
        $$
    - 对于图像到文本的方向，定义预测的概率分布：
        $$
        p_{ij}=\frac{\exp(s_{ij})}{\sum_{k=1}^N\exp(s_{ik})}
        $$
        - 相应的损失函数为：$\mathcal L_{\text{img2txt}}=-\frac1N\sum_{i=1}^N\log p_{i,i}$
    - 对于文本到图像的方向，定义预测的概率分布为：
        $$
        q_{ij}=\frac{\exp(s_{ij})}{\sum_{k=1}^N\exp(s_{k,j})}
        $$
        - 相应的损失函数为：$\mathcal L_{\text{txt2img}}=-\frac1N\sum_{i=1}^N\log q_{i,i}$
    - 总的损失
        $$
        \mathcal L=\frac12({L_{\text{img2txt}}+L_{\text{txt2img}}})
        $$ 

### reward modeling

- 而 instruct GPT 开创的 reward modeling 追求的是学习 reward function，即给定一个 input, response pair，输出一个好坏的分数（scalar output）
    $$
    \text{loss}(\theta) = -\frac{1}{\left(\frac{K}{2}\right)} \mathbb{E}_{(x, y_w, y_l) \sim \mathcal{D}} \left[ \log \left( \sigma \left( r_\theta(x, y_w) - r_\theta(x, y_l) \right) \right) \right]
    $$ 
- 对比学习，强调的是 $f(I_i),g(T_i)$ 将原始的输入，映射到 hidden space，得到的 feature，即为表征学习（representation learning）；

## constrastive loss

### (pairwise) Margin Ranking loss

$$
L(y_w,y_l)=\max(0, \delta-(R(y_w) - R(y_l))
$$

- 也就是说，目标追求的是 $R(y_w)-R(y_l)\geq \delta$ 时，loss 才为0；

    $$
    R(y_w)-R(y_l)\geq \delta\\
    R(y_w)\geq R(y_l)+\delta
    $$
    
    - Reward model 对 $y_w$ 的打分要比对 $y_l$ 的打分 + $\delta$ 还多；